In this section we are using ensemble method where we will be combining both CNN and LSTM and will be appling with these specification:

1.   Activation Funtion: Sigmoid
2.   Optimizer: NADAM


*   cause these were the ones who gave maximum accuracy for out dataset.





In [ ]:
import pickle
import pandas as pd
from urllib.request import urlopen
# from IPython.display import display
url = "https://springernature.figshare.com/ndownloader/files/34135113"
response = urlopen(url)
with response as file:
    dataset = pickle.load(file)

df=pd.DataFrame(dataset)


import numpy as np
df['signal_pleth'] = df['signal_pleth'].apply(lambda x: np.mean(x))
df['signal_ecg_i'] = df['signal_ecg_i'].apply(lambda x: np.mean(x))
df['signal_ecg_ii'] = df['signal_ecg_ii'].apply(lambda x: np.mean(x))
df['signal_ecg_iii'] = df['signal_ecg_iii'].apply(lambda x: np.mean(x))
df['PSG_Abdomen'] = df['PSG_Abdomen'].apply(lambda x: np.mean(x))
df['PSG_Flow'] = df['PSG_Flow'].apply(lambda x: np.mean(x))
df['PSG_Position'] = df['PSG_Position'].apply(lambda x: np.mean(x))
df['PSG_Snore'] = df['PSG_Snore'].apply(lambda x: np.mean(x))
df['PSG_Thorax'] = df['PSG_Thorax'].apply(lambda x: np.mean(x))


df.drop(['patient', 'timestamp_datetime'], axis=1)

df = df.dropna()
df['anomaly'] = df['anomaly'].astype(int)

<ipython-input-1-7402885108f7>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['anomaly'] = df['anomaly'].astype(int)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['event_encoded'] = label_encoder.fit_transform(df['event'])

# Calculate the number of unique classes
num_classes = df['event_encoded'].nunique()

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    df[['HR(bpm)', 'SpO2(%)', 'PI(%)', 'RR(rpm)', 'PVCs(/min)', 'anomaly', 'signal_pleth', 'signal_ecg_i', 'signal_ecg_ii', 'signal_ecg_iii', 'PSG_Abdomen', 'PSG_Flow', 'PSG_Position', 'PSG_Snore', 'PSG_Thorax']],
    df['event_encoded'],
    test_size=0.2
)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5)

# Reshape the data for CNN (assuming you have 1D data)
input_shape = (X_train.shape[1], 1)

# Hybrid Method

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Embedding
from keras.optimizers import Nadam
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score

# Load your dataset and preprocess it as needed
# Assuming you have X_train, X_val, X_test, y_train, y_val, y_test

# Train CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(32, kernel_size=3, activation='sigmoid', input_shape=(X_train.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='sigmoid'))
cnn_model.add(Dense(num_classes, activation='softmax'))

cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer=Nadam(learning_rate=0.001), metrics=['accuracy'])
cnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)




Epoch 1/10
19766/19766 [==============================] - 56s 3ms/step - loss: 0.1706 - accuracy: 0.9281 - val_loss: 0.1496 - val_accuracy: 0.9357
Epoch 2/10
19766/19766 [==============================] - 55s 3ms/step - loss: 0.1426 - accuracy: 0.9379 - val_loss: 0.1412 - val_accuracy: 0.9392
Epoch 3/10
19766/19766 [==============================] - 54s 3ms/step - loss: 0.1357 - accuracy: 0.9421 - val_loss: 0.1362 - val_accuracy: 0.9417
Epoch 4/10
19766/19766 [==============================] - 53s 3ms/step - loss: 0.1316 - accuracy: 0.9442 - val_loss: 0.1380 - val_accuracy: 0.9404
Epoch 5/10
19766/19766 [==============================] - 53s 3ms/step - loss: 0.1285 - accuracy: 0.9459 - val_loss: 0.1308 - val_accuracy: 0.9452
Epoch 6/10
19766/19766 [==============================] - 54s 3ms/step - loss: 0.1255 - accuracy: 0.9472 - val_loss: 0.1288 - val_accuracy: 0.9466
Epoch 7/10
19766/19766 [==============================] - 54s 3ms/step - loss: 0.1230 - accuracy: 0.9482 - val_loss: 0

In [ ]:
# Train LSTM model
lstm_model = Sequential()

# Reshape the data for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


lstm_model.add(LSTM(64, activation='sigmoid', return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(64, activation='sigmoid'))
lstm_model.add(Dense(num_classes, activation='softmax'))

lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer=Nadam(learning_rate=0.001), metrics=['accuracy'])
lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)


# Make predictions with both models
cnn_predictions = cnn_model.predict(X_test)
lstm_predictions = lstm_model.predict(X_test)

# Ensemble the predictions using simple averaging
ensemble_predictions = (cnn_predictions + lstm_predictions) / 2

# Convert predictions to class labels
ensemble_labels = np.argmax(ensemble_predictions, axis=1)

# Calculate accuracy of the ensemble model
ensemble_accuracy = accuracy_score(y_test, ensemble_labels)
print(f'Ensemble Model Accuracy: {ensemble_accuracy}')

Epoch 1/10
19766/19766 [==============================] - 369s 18ms/step - loss: 0.2347 - accuracy: 0.9144 - val_loss: 0.1748 - val_accuracy: 0.9249
Epoch 2/10
19766/19766 [==============================] - 354s 18ms/step - loss: 0.1618 - accuracy: 0.9316 - val_loss: 0.1598 - val_accuracy: 0.9335
Epoch 3/10
19766/19766 [==============================] - 358s 18ms/step - loss: 0.1503 - accuracy: 0.9367 - val_loss: 0.1573 - val_accuracy: 0.9330
Epoch 4/10
19766/19766 [==============================] - 355s 18ms/step - loss: 0.1427 - accuracy: 0.9397 - val_loss: 0.1408 - val_accuracy: 0.9404
Epoch 5/10
19766/19766 [==============================] - 353s 18ms/step - loss: 0.1351 - accuracy: 0.9428 - val_loss: 0.1366 - val_accuracy: 0.9421
Epoch 6/10
19766/19766 [==============================] - 351s 18ms/step - loss: 0.1302 - accuracy: 0.9453 - val_loss: 0.1333 - val_accuracy: 0.9442
Epoch 7/10
19766/19766 [==============================] - 354s 18ms/step - loss: 0.1267 - accuracy: 0.9473